In [193]:
import csv
import tf2onnx
import onnx

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [194]:
dataset = 'model/handClassifier/keypoint.csv'
model_save_path = 'model/handClassifier/keypoint_classifier.keras'
tflite_save_path = 'model/handClassifier/keypoint_classifier.tflite'

# Set number of classes

In [195]:
NUM_CLASSES = 6

# Dataset reading

In [196]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [197]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [198]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

# Model building

In [199]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [200]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_24 (Dropout)            │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,136 (4.44 KB)

 Trainable params: 1,136 (4.44 KB)

 Non-trainable params: 0 (0.00 B)

In [201]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [202]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [203]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.1797 - loss: 2.0038
Epoch 1: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1787 - loss: 1.9409 - val_accuracy: 0.3080 - val_loss: 1.7207
Epoch 2/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2422 - loss: 1.7577
Epoch 2: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2948 - loss: 1.7168 - val_accuracy: 0.4350 - val_loss: 1.6042
Epoch 3/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3984 - loss: 1.6160
Epoch 3: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3661 - loss: 1.6050 - val_accuracy: 0.4977 - val_loss: 1.4740
Epoch 4/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3125 - loss: 1.5330
Epoch 4: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━

In [204]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9501 - loss: 0.2983 


In [205]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [206]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[0.11151804 0.55406773 0.236554   0.01870265 0.07140958 0.00774793]
1


# Convert to model for Tensorflow-Lite

In [207]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [208]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmp6wfjgsd7\assets


INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmp6wfjgsd7\assets


Saved artifact at 'C:\Users\narte\AppData\Local\Temp\tmp6wfjgsd7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_12')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2230449420048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230449421584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230449421392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230449411408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230449414288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2230449413328: TensorSpec(shape=(), dtype=tf.resource, name=None)


6660

# Inference test

In [209]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [210]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [211]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [212]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [213]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[0.11151804 0.55406773 0.23655398 0.01870266 0.0714096  0.00774793]
1


In [ ]:
input_signature = [tf.TensorSpec([1, 42], tf.float32, name="input")]
model.output_names = ["output"]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)
onnx.save(onnx_model, "model/handClassifier/model.onnx")
